# Query database over an SSH tunnel
Blog post related to this notebook is here:
https://medium.com/@amirziai/query-your-database-over-an-ssh-tunnel-with-pandas-603ce49b35a1#.4dzdcrl48

## SSH and database information

In [3]:
# ssh variables
host = '199.87.187.72'
localhost = '127.0.0.1'
ssh_username = 'tunnel'
ssh_password = 'paleocake'

In [21]:
# database variables
user='alina'
password='alinarocks'


## Install sshtunnel

In [9]:
!pip install sshtunnel
!pip install mysqlclient

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.
     |████████████████████████████████| 88 kB 370 kB/s eta 0:00:01
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp38-cp38-macosx_10_9_x86_64.whl size=56306 sha256=9e943d8bf38d495559ed48aaf3eb7fb7cb52f465598a07eeec979c5a425bbef1
  Stored in directory: /Users/younglihong/Library/Caches/pip/wheels/3a/c1/c3/5a19639a551c921c2c2b39468f4278ce5aa27b4e386a4158e4
Successfully built mysqlclient
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


## Import packages

In [10]:
from sshtunnel import SSHTunnelForwarder
import MySQLdb as db
import pandas as pd

## The query function

In [22]:
def query(q, database):
    with SSHTunnelForwarder(
        (host, 22),
        ssh_username=ssh_username,
        ssh_password=ssh_password,
        remote_bind_address=(localhost, 3306)
    ) as server:
        conn = db.connect(host=localhost,
                               port=server.local_bind_port,
                               user=user,
                               passwd=password,
                               db=database)

        return pd.read_sql_query(q, conn)

## Query time

In [ ]:
#Fill appropriate state number and fips code for county
STATE_NO = ''
FIPS_CODE = ''
qd = 'select * from deed_'+ STATE_NO +' where FIPS_CODE == ' + FIPS_CODE
qo = 'select * from ols_'+ STATE_NO +' where FIPS_CODE == ' + FIPS_CODE
qa = 'select * from tax_'+ STATE_NO +' where FIPS_CODE == ' + FIPS_CODE

In [23]:
deed_df = query(qd, 'corelogic_deed')
ols_df = query(qd, 'corelogic_ols')
tax_df = query(qd, 'corelogic_tax')

In [24]:
deed_df.head()

,FIPS_CODE,APN_PARCEL_NUMBER_UNFORMATTED,APN_SEQUENCE_NUMBER,COMPOSITE_PROPERTY_LINKAGE_KEY,ORIGINAL_APN,TAX_ACCOUNT_NUMBER,ONLINE_FORMATTED_PARCEL_ID,COMPOSITE_TRANSACTION_ID,TRANSACTION_BATCH_DATE,TRANSACTION_BATCH_SEQUENCE_NUMBER,...,PRINCIPAL_1_FULL_NAME,PRINCIPAL_1_LAST_NAME,PRINCIPAL_1_FIRST_NAME,PRINCIPAL_1_MIDDLE_INITIAL,PRINCIPAL_1_TITLE_,PRINCIPAL_2_FULL_NAME,PRINCIPAL_2_LAST_NAME,PRINCIPAL_2_FIRST_NAME,PRINCIPAL_2_MIDDLE_INITIAL,PRINCIPAL_2_TITLE
0,5001,00100154000,1,0500100100154000 ...,001-00154-000,,001-00154-000,050011930010100001000,1930-01-01,1,...,,,,,,,,,,
1,5001,,0,,,,,050011930010100003000,1930-01-01,3,...,,,,,,,,,,
2,5001,00100127000,1,0500100100127000 ...,001-00127-000,,001-00127-000,050011930010100004000,1930-01-01,4,...,,,,,,,,,,
3,5001,,0,,,,,050011930010100005000,1930-01-01,5,...,,,,,,,,,,
4,5001,,0,,,,,050011930010100006000,1930-01-01,6,...,,,,,,,,,,


In [ ]:
ols_df.head()

In [ ]:
tax_df.head()

In [ ]:
#Fill county name
county_name = ''
name_deed= 'deed_'+ STATE_NO + '_' + county_name + '.csv'
name_ols= 'ols_'+ STATE_NO + '_' + county_name + '.csv'
name_tax= 'tax_'+ STATE_NO + '_' + county_name + '.csv'

In [ ]:
deed_df.to_csv(name_deed)
ols_df.to_csv(name_ols)
tax_df.to_csv(name_tax)